In [1]:
import requests
import uuid
import time
import json

with open('./data/api_url.txt','r') as api_url : 
    api_url = api_url.readline()

with open('./data/secret_key.txt','r') as secret_key : 
    secret_key = secret_key.readline()

image_file = './data/20211019_172952.png'
output_file = './output.json'

request_json = {
    'images': [
        {
            'format': 'png',
            'name': 'demo',
            'templateIds': [11112]
        }
    ],
    'requestId': str(uuid.uuid4()),
    'version': 'V2',
    'timestamp': int(round(time.time() * 1000))
}

payload = {'message': json.dumps(request_json).encode('UTF-8')}
files = [
  ('file', open(image_file,'rb'))
]
headers = {
  'X-OCR-SECRET': secret_key
}

response = requests.request("POST", api_url, headers=headers, data = payload, files = files)

res = json.loads(response.text.encode('utf8'))
print(res)

with open(output_file, 'w', encoding='utf-8') as outfile:
    json.dump(res, outfile, indent=4, ensure_ascii=False)

{'version': 'V2', 'requestId': '619d9073-ba72-495b-beeb-4ed59afb5215', 'timestamp': 1634688284227, 'images': [{'uid': '7eb603d8ea46478197304c71cb87bf6c', 'name': 'demo', 'inferResult': 'SUCCESS', 'message': 'SUCCESS', 'matchedTemplate': {'id': 11112, 'name': '코로나19명단'}, 'validationResult': {'result': 'NO_REQUESTED'}, 'title': {'name': '스캔버전', 'boundingPoly': {'vertices': [{'x': 102.0, 'y': 80.0}, {'x': 493.0, 'y': 80.0}, {'x': 493.0, 'y': 120.0}, {'x': 102.0, 'y': 120.0}]}, 'inferText': '코로나19 대응 출입자 명부', 'inferConfidence': 0.99995, 'subFields': [{'boundingPoly': {'vertices': [{'x': 102.0, 'y': 83.0}, {'x': 238.0, 'y': 83.0}, {'x': 238.0, 'y': 120.0}, {'x': 102.0, 'y': 120.0}]}, 'inferText': '코로나19', 'inferConfidence': 1.0, 'lineBreak': False}, {'boundingPoly': {'vertices': [{'x': 246.0, 'y': 81.0}, {'x': 313.0, 'y': 81.0}, {'x': 313.0, 'y': 120.0}, {'x': 246.0, 'y': 120.0}]}, 'inferText': '대응', 'inferConfidence': 1.0, 'lineBreak': False}, {'boundingPoly': {'vertices': [{'x': 320.0, 'y

In [19]:
# 날짜
date = res['images'][0]['fields'][0]['inferText'].split('\n')

# 시간
time = res['images'][0]['fields'][1]['inferText'].split('\n')

# 수집동의
checkbox = res['images'][0]['fields'][2]['inferText'].split('\n')

# 시군구
sigungu = res['images'][0]['fields'][3]['inferText'].split('\n')

# 전화번호
tel = res['images'][0]['fields'][4]['inferText'].split('\n')

# 비고
extra = res['images'][0]['fields'][5]['inferText'].split('\n')

In [20]:
# max length 
length = 0 
li = [date, time, checkbox, sigungu, tel, extra]
for i in li : 
    length = max(len(i), length)


# 패딩 맞춰주기 
import numpy as np 
df_li = []
for i in li :
     df_li.append(np.pad(i, (0,length - len(i)), 'constant', constant_values=0))

In [21]:
import pandas as pd
df = pd.DataFrame(df_li).transpose()
df.columns = ['date', 'time', 'checkbox', 'sigungu', 'tel', 'extra']
df

,date,time,checkbox,sigungu,tel,extra
0,10/18,10: 30,v,용산구,010-1256-7910,
1,10/18,10: 40,V,서초구,010-3491-2376,0
2,10/ 18,10:42,v,강남구,010-2233-4455,0
3,10/18,10:50,V,영등포구,010-3712-2195,0
4,10/18,11:30,v,송파구,010-9930-1393,0
5,10/ 18,11:40,V,마포구,010-1311-4897,0
6,10/18,13:30,v,성북구,010-3158-7762,0
7,10/18,15:20,0,종로구,010-1783-4461,0
8,10/19,10:00,0,용산구,010-0127-9903,0
9,10/19,11:11,0,강남구,010-1018-1428,0


In [29]:
import warnings
warnings.filterwarnings('ignore')

# 공백 및 특수문자 제거 
for col in ['date', 'time', 'checkbox', 'sigungu', 'tel', 'extra'] : 
    df[col]= df[col].str.replace(' ', '')
    df[col]= df[col].str.replace('.', '')

# 체크박스 형식 통일
df.loc[(df['checkbox']=='v'), 'checkbox'] = 'V'

df

,date,time,checkbox,sigungu,tel,extra
0,10/18,10:30,V,용산구,010-1256-7910,
1,10/18,10:40,V,서초구,010-3491-2376,0
2,10/18,10:42,V,강남구,010-2233-4455,0
3,10/18,10:50,V,영등포구,010-3712-2195,0
4,10/18,11:30,V,송파구,010-9930-1393,0
5,10/18,11:40,V,마포구,010-1311-4897,0
6,10/18,13:30,V,성북구,010-3158-7762,0
7,10/18,15:20,0,종로구,010-1783-4461,0
8,10/19,10:00,0,용산구,010-0127-9903,0
9,10/19,11:11,0,강남구,010-1018-1428,0


In [30]:
df.to_csv('example.csv')